# Data extraction

In [3]:
import yfinance as yf

tickers = yf.Tickers("AAPL")
data = tickers.history(period="1mo")
closing_price = data['Close']
print(closing_price)



[*********************100%***********************]  1 of 1 completed

Ticker            AAPL
Date                  
2025-04-21  192.907028
2025-04-22  199.478424
2025-04-23  204.332062
2025-04-24  208.097107
2025-04-25  209.005920
2025-04-28  209.864792
2025-04-29  210.933395
2025-04-30  212.221710
2025-05-01  213.040634
2025-05-02  205.081070
2025-05-05  198.629532
2025-05-06  198.250015
2025-05-07  195.992981
2025-05-08  197.231369
2025-05-09  198.270004
2025-05-12  210.789993
2025-05-13  212.929993
2025-05-14  212.330002
2025-05-15  211.449997
2025-05-16  211.259995
2025-05-19  208.779999
2025-05-20  206.860001
2025-05-21  202.050003


# Feature Construction